In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']




Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [4]:
task='GlobalLocal'

conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskg with parameters: {'BIDS_events': 'Stim

In [5]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [6]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [7]:
# Assuming you have a list of subjects
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"

In [8]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF

let's manually make the occ filtered sig electrodes

In [9]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [10]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [11]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [12]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 164
Total number of significant lpfc electrodes across all subjects: 59
Total number of v1 electrodes across all subjects: 19
Total number of significant v1 electrodes across all subjects: 11
Total number of occ electrodes across all subjects: 76
Total number of significant occ electrodes across all subjects: 51
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [13]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [14]:
# Example call to the function (you need to replace the arguments with actual data)
# dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


some helper functions

In [115]:
def get_max_trials_per_condition(
    subjects_mne_objects, condition_names, subjects,
    sig_electrodes_per_subject_roi, roi, obs_axs
):
    """
    Find the maximum number of trials per condition across all subjects for a given ROI.
    """
    max_trials_per_condition = {condition: 0 for condition in condition_names}

    for sub in subjects:
        sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        if not sig_electrodes:
            continue
        for condition_name in condition_names:
            epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
            epochs_data = epochs.get_data(copy=True)
            n_trials = epochs_data.shape[obs_axs - 1]
            max_trials_per_condition[condition_name] = max(
                max_trials_per_condition[condition_name], n_trials
            )
    return max_trials_per_condition

def make_subject_labeled_array(
    sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
    roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
):
    """
    Process data for a subject in a given ROI.

    This function performs the following steps for the given subject:
    1. Retrieves the significant electrodes in the specified ROI for the subject.
    2. For each condition:
        a. Extracts the epoch data for the significant electrodes.
        b. Randomizes the trial order using the provided random state.
        c. Pads the data with NaNs to match the maximum number of trials for that condition.
    3. Collects the processed data for all conditions into a dictionary.
    4. Creates a LabeledArray from the processed data, assigning appropriate channel and time labels.

    Returns a LabeledArray for the subject.
    """
    sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
    if not sig_electrodes:
        return None

    subject_nested_dict = {}

    # Get channel names for this subject's ROI
    sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

    # Loop through each condition
    for condition_name in condition_names:
        # Extract the epoch data for the current condition and subject
        epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
        epochs_data = epochs.get_data(copy=True)

        # Randomize the trial order
        n_trials = epochs_data.shape[obs_axs - 1]
        print(f'in roi {roi}, subject {sub} has {n_trials} trials for condition {condition_name}')
        trial_indices = np.arange(n_trials)
        rng.shuffle(trial_indices)
        epochs_data = epochs_data.take(trial_indices, axis=obs_axs - 1)

        # Get the target number of trials for padding
        max_trials = max_trials_per_condition[condition_name]

        # Pad with NaNs if necessary
        if n_trials < max_trials:
            padded_shape = list(epochs_data.shape)
            padded_shape[obs_axs - 1] = max_trials
            padded_data = np.full(padded_shape, np.nan)
            indexer = [slice(None)] * epochs_data.ndim
            indexer[obs_axs - 1] = slice(0, n_trials)
            padded_data[tuple(indexer)] = epochs_data
        else:
            padded_data = epochs_data

        subject_nested_dict[condition_name] = padded_data

    # Get time labels
    times = epochs.times
    str_times = [str(time) for time in times]
    np_array_str_times = np.array(str_times)

    # Create a LabeledArray for the subject
    subject_labeled_array = create_subject_labeled_array_from_dict(
        subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
    )

    return subject_labeled_array

def create_subject_labeled_array_from_dict(
    subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
):
    """
    Create a LabeledArray for a subject with given data and labels.
    """
    subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
    subject_labeled_array.labels[chans_axs].values = sub_channel_names  # Channels axis
    subject_labeled_array.labels[time_axs].values = np_array_str_times  # Time axis
    return subject_labeled_array

def concatenate_subject_labeled_arrays(
    roi_labeled_array, subject_labeled_array, chans_axs
):
    """
    Concatenate a subject's LabeledArray to the ROI's LabeledArray along the channels axis.
    """
    if roi_labeled_array is None:
        return subject_labeled_array
    else:
        return roi_labeled_array.concatenate(subject_labeled_array, axis=chans_axs)

main function

In [116]:
def put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects, condition_names, rois, subjects,
    sig_electrodes_per_subject_roi, obs_axs=1, chans_axs=2, time_axs=3,
    random_state=None
):
    """
    Organize the MNE data into separate LabeledArrays for each ROI and subject,
    with randomized trial ordering within each subject before concatenation.
    Concatenates subject data along the channels axis.

    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary mapping ROIs to subjects and their corresponding electrodes.
    - obs_axs: The trials dimension (1-based indexing).
    - chans_axs: The channels dimension (1-based indexing).
    - time_axs: The time dimension (1-based indexing).
    - random_state: Optional; an integer seed, NumPy RandomState, or None for random shuffling.

    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
                          Each LabeledArray has dimensions: [Conditions, Trials, Channels, Timepoints]
    """
    # Set up the random state
    rng = np.random.RandomState(random_state)
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        # First pass: Find the max number of trials per condition across all subjects
        max_trials_per_condition = get_max_trials_per_condition(
            subjects_mne_objects, condition_names, subjects,
            sig_electrodes_per_subject_roi, roi, obs_axs
        )

        # Initialize the ROI LabeledArray
        roi_labeled_array = None

        # Second pass: Process each subject's data
        for sub in subjects:
            subject_labeled_array = make_subject_labeled_array(
                sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
                roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
            )
            if subject_labeled_array is None:
                continue  # Skip if subject has no data for this ROI

            # Concatenate subject's data into the ROI LabeledArray
            roi_labeled_array = concatenate_subject_labeled_arrays(
                roi_labeled_array, subject_labeled_array, chans_axs
            )

        # Add the concatenated LabeledArray to the ROI dictionary
        if roi_labeled_array is not None:
            roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays

In [117]:
# Example usage:
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects,
    condition_names,
    rois,
    subjects,
    sig_electrodes_per_subject_roi,
    obs_axs=1,  # Trials dimension (1-based indexing)
    chans_axs=2,  # Channels dimension (1-based indexing)
    time_axs=3,   # Time dimension (1-based indexing)
    random_state=42  # For reproducibility
)

in roi lpfc, subject D0057 has 59 trials for condition Stimulus/BigLetters/SmallLetterh/Taskg
in roi lpfc, subject D0057 has 54 trials for condition Stimulus/BigLetters/SmallLetterh/Taskl
in roi lpfc, subject D0057 has 53 trials for condition Stimulus/BigLetters/SmallLetters/Taskg
in roi lpfc, subject D0057 has 34 trials for condition Stimulus/BigLetters/SmallLetters/Taskl
in roi lpfc, subject D0057 has 51 trials for condition Stimulus/BigLetterh/SmallLetterh/Taskg
in roi lpfc, subject D0057 has 60 trials for condition Stimulus/BigLetterh/SmallLetterh/Taskl
in roi lpfc, subject D0057 has 42 trials for condition Stimulus/BigLetterh/SmallLetters/Taskg
in roi lpfc, subject D0057 has 52 trials for condition Stimulus/BigLetterh/SmallLetters/Taskl
in roi lpfc, subject D0059 has 48 trials for condition Stimulus/BigLetters/SmallLetterh/Taskg
in roi lpfc, subject D0059 has 57 trials for condition Stimulus/BigLetters/SmallLetterh/Taskl
in roi lpfc, subject D0059 has 62 trials for condition Stimu

In [105]:
roi_labeled_arrays['lpfc'].shape

(8, 64, 59, 641)

remove trials with nans from roi labeled arrays if just getting the minimum number of trials for all subjects

In [108]:
def remove_nans_from_labeled_array(labeled_array, obs_axs=1, chans_axs=2, time_axs=3):
    """
    Remove trials that have NaN values from a LabeledArray.

    Parameters:
    - labeled_array: A LabeledArray where the 0th axis labels are conditions, 1st axis labels are trials, 
                     2nd axis labels are channels, and 3rd axis labels are timepoints.
    - obs_axs: The trials dimension (1-based indexing).
    - chans_axs: The channels dimension (1-based indexing).
    - time_axs: The time dimension (1-based indexing).

    Returns:
    - labeled_array_no_nans: A LabeledArray with only trials that have no NaN values.
    """
    # Convert to zero-based indexing
    obs_axs -= 1
    chans_axs -= 1
    time_axs -= 1

    # Initialize a dictionary to store data without NaNs for each condition
    reshaped_data_dict = {}

    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name] # Shape: (Trials, Channels, Timepoints)

        # Find the indices of trials that do not contain NaNs
        # Reduce over channels and time axes to check if any NaN exists in a trial
        valid_trial_indices = ~np.isnan(condition_data).any(axis=(chans_axs, time_axs))

        # valid_trial_indices is a boolean array of shape (Trials,)
        # Select only the valid trials
        condition_data_clean = condition_data[valid_trial_indices, :, :]

        # Store the processed data for this condition
        reshaped_data_dict[condition_name] = condition_data_clean

    # Create a new LabeledArray from the reshaped data dictionary
    labeled_array_no_nans = LabeledArray.from_dict(reshaped_data_dict)

    return labeled_array_no_nans

def remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=1, chans_axs=2, time_axs=3):
    """
    Loop through all ROIs and apply the NaN removal function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - obs_axs: The trials dimension (1-based indexing).
    - chans_axs: The channels dimension (1-based indexing).
    - time_axs: The time dimension (1-based indexing).

    Returns:
    - labeled_arrays_no_nans: Dictionary where keys are ROIs and values are LabeledArrays with NaNs removed.
    """
    roi_labeled_arrays_no_nans = {}
    
    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the NaN removal function to the current labeled array
        labeled_array_no_nans = remove_nans_from_labeled_array(
            labeled_array, obs_axs=obs_axs, chans_axs=chans_axs, time_axs=time_axs
        )

        # Store the reshaped data for this ROI
        roi_labeled_arrays_no_nans[roi] = labeled_array_no_nans

    return roi_labeled_arrays_no_nans

In [120]:
roi_labeled_arrays_no_nans = remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=1, chans_axs=2, time_axs=3)
roi_labeled_arrays_no_nans['lpfc']

array([[[[-3.04644763e-01, -4.27678347e-01, -4.97747958e-01, ...,
          -1.00266480e+00, -1.08392704e+00, -1.14953256e+00],
         [-1.04423034e+00, -1.01916778e+00, -1.01196134e+00, ...,
           1.71241403e+00,  1.47226834e+00,  1.34279430e+00],
         [ 2.72478282e-01,  2.77666748e-01,  2.49155253e-01, ...,
          -3.45492065e-01, -3.30436707e-01, -3.06903750e-01],
         ...,
         [ 1.80977666e+00,  1.44853127e+00,  1.10428512e+00, ...,
          -2.60263056e-01, -1.99117616e-01, -1.37402713e-01],
         [-4.53894764e-01, -4.19212759e-01, -3.50485176e-01, ...,
          -1.52206630e-01, -1.01805553e-01,  2.50967331e-02],
         [-1.67640612e-01, -2.16709301e-01, -2.95076489e-01, ...,
           1.13752592e+00,  1.20565796e+00,  1.28719842e+00]],

        [[ 1.87286711e+00,  1.73476148e+00,  1.61565661e+00, ...,
           4.61819649e-01,  6.14762008e-01,  7.47779608e-01],
         [-1.35569125e-01, -3.05584520e-01, -4.93779093e-01, ...,
           4.53342050e

In [69]:
'''
11/6 need to update the obs_axs here to work with obs_axs=1 and also all the way down too 
(though I'm not sure why the obs_axs is 0 for decoding, maybe cuz I make arrays for each condition?)
'''
def concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find, obs_axs=1):
    """
    Concatenate trials across condition names that contain specific strings.
    Assign labels based on the groupings of the conditions.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - roi: The specific ROI to process.
    - strings_to_find: List of strings or list of lists of strings to search for in condition names.
                      If a list of strings is provided, each string is treated as its own condition group.
                      If a list of lists is provided, each sublist represents a group of conditions.
    - obs_axs (int) : The trials dimension. Concatenation will happen along this axis. This is the 1st dimension (not 0th) because conditions in the labeled array is the 0th. But we will subtract 1 if not considering conditions as a dimension (looping over conditions)
    
    Returns:
    - concatenated_data: The concatenated trials by (channels, timepoints, or whatever your other dimensions are) across the matching conditions.
    - labels: A numpy array of labels (0, 1, 2, ...) corresponding to each group of conditions.
    - cats: Dictionary of {condition_name: index} for decoding.
    """
    concatenated_data = []
    labels = []
    cats = {}

    # subtract 1 from the obs_axs index because we're not considering conditions as a dimension
    obs_axs -= 1
    # Track current label index
    current_label = 0

    # Normalize strings_to_find so each entry is a list (whether it's a string or a list of strings)
    if isinstance(strings_to_find, list) and all(isinstance(s, str) for s in strings_to_find):
        # If it's a flat list of strings, convert each string into its own single-item list
        strings_to_find = [[s] for s in strings_to_find]

    # Iterate over each group (whether it's a single string or a list of strings)
    for string_group in strings_to_find:
        # Find condition names that match any of the strings in the current string_group
        matching_conditions = [cond for cond in roi_labeled_arrays[roi].keys() if any(s in cond for s in string_group)]

        if not matching_conditions:
            continue

        # Concatenate data for all matching conditions
        data_to_concatenate = []
        for cond in matching_conditions:
            # Extract data for the current condition
            data = roi_labeled_arrays[roi][cond]  # Shape: (trials, channels, timepoints)
            data_to_concatenate.append(data)
            
            # Update labels for the current condition group
            labels.extend([current_label] * data.shape[0])
        
        # Check if we have data to concatenate for this condition group
        if data_to_concatenate:
            concatenated_data.append(np.concatenate(data_to_concatenate, axis=obs_axs))

        # Assign current label to the condition group (based on the first string in the group for reference)
        cats[tuple(string_group)] = current_label
        current_label += 1

    # Ensure there is data to concatenate
    if not concatenated_data:
        raise ValueError(f"No matching conditions found for ROI: {roi} and strings: {strings_to_find}")

    # Concatenate all condition data along the trials axis
    concatenated_data = np.concatenate(concatenated_data, axis=obs_axs)
    
    return concatenated_data, np.array(labels), cats

set up decoding output directory and conditions to compare 9/12

In [70]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs')

if conditions == stimulus_experiment_conditions:
    condition_comparisons = {}
    condition_comparisons['congruency'] = [['c25.0', 'c75.0'], ['i25.0', 'i75.0']]
    condition_comparisons['switchType'] = [['r25.0', 'r75.0'], ['s25.0', 's75.0']]
    
elif conditions == stimulus_conditions:
    condition_comparisons = {}
    condition_comparisons['BigLetter'] = ['BigLetters', 'BigLetterh']
    condition_comparisons['SmallLetter'] = ['SmallLetters', 'SmallLetterh']
    condition_comparisons['Task'] = ['Taskg', 'Taskl']
    
    # strings_to_find = ['BigLetters', 'BigLetterh']
    # condition_0_name = 'BigLetters'
    # condition_1_name = 'BigLetterh'

    # strings_to_find = ['SmallLetters', 'SmallLetterh']
    # condition_0_name = 'SmallLetters'
    # condition_1_name = 'SmallLetterh'

    # strings_to_find = ['Taskg', 'Taskl']
    # condition_0_name = 'Taskg'
    # condition_1_name = 'Taskl'

do decoding on the reshaped time indexed arrays 9/12

let's try a class for storing and plotting the decoding scores for each roi 9/13

In [24]:
class ROIScoreData:
    def __init__(self, roi, bin_size):
        self.roi = roi
        self.bin_size = bin_size
        self.bin_centers = []
        self.accuracy_scores = []
        self.precision_scores = []
        self.recall_scores = []
        self.f1_scores = []
        self.d_prime_scores = []

    def add_scores(self, bin_center, accuracy, precision, recall, f1, d_prime):
        self.bin_centers.append(bin_center)
        self.accuracy_scores.append(accuracy)
        self.precision_scores.append(precision)
        self.recall_scores.append(recall)
        self.f1_scores.append(f1)
        self.d_prime_scores.append(d_prime)

    def plot_scores(self, strings_to_find, save_dir):
        plt.figure(figsize=(10, 6))

        # Plot accuracy
        plt.plot(self.bin_centers, self.accuracy_scores, label='Accuracy', marker='o')

        # # Plot precision
        # plt.plot(self.bin_centers, self.precision_scores, label='Precision', marker='o')

        # # Plot recall
        # plt.plot(self.bin_centers, self.recall_scores, label='Recall', marker='o')

        # # Plot f1 score
        # plt.plot(self.bin_centers, self.f1_scores, label='F1 Score', marker='o')

        # Plot d-prime
        plt.plot(self.bin_centers, self.d_prime_scores, label='D-Prime', marker='o')

        # Set plot details
        plt.xlabel('Time Bin Centers (s)')
        plt.ylabel('Score')
        plt.title(f'{condition_0_name} vs. {condition_1_name} Decoding Over Time for ROI: {self.roi}')
        plt.legend()
        plt.grid(True)
        plt.ylim(0,1)

        # Save the figure
        file_name = f'{self.roi}_{condition_0_name}_vs_{condition_1_name}_decoding_over_time_dprime_acc.png'
        plt.savefig(os.path.join(save_dir, file_name))
        plt.close()


In [83]:
def get_and_plot_confusion_matrix_for_rois_jim(roi_labeled_arrays, rois, condition_comparison, strings_to_find, save_dir, time_interval_name=None, n_splits=5, n_repeats=5, obs_axs=0):
    """
    Compute the confusion matrix for each ROI and return it. Does this using all time points for training.

    Parameters:
    - roi_labeled_arrays: Dictionary containing the reshaped data for each ROI.
    - rois: List of regions of interest (ROIs) to process.
    - condition_comparison: the condition that we're comparing labels for (i.e., BigLetter)
    - strings_to_find: List of strings or string groups to identify condition labels.
    - save_dir: Directory to save the confusion matrix plots.
    - time_interval_name: Optional string to add to the filename for the time interval (e.g., 'pre_stimulus' or 'post_stimulus').
    - obs_axs: the trials axis

    Returns:
    - confusion_matrices: Dictionary containing confusion matrices for each ROI.
    """
    confusion_matrices = {}

    # reshaped_roi_labeled_arrays_for_decoding = reshape_all_rois_for_decoding(roi_labeled_arrays)

    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Concatenate the trials and get labels
        # concatenated_data, labels, cats = concatenate_conditions_by_string(reshaped_roi_labeled_arrays_for_decoding, roi, strings_to_find)
        concatenated_data, labels, cats = concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find, obs_axs) # don't reshape

        print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")
        
        # Calculate trial and nan trial counts per condition
        trial_counts = {}
        nan_counts_per_channel = {}

        for string_group in strings_to_find:
            # Get the label associated with the current string group
            condition_label = cats[tuple(string_group) if isinstance(string_group, list) else (string_group,)]
            
            # Select trials for the current condition group
            condition_trials = labels == condition_label
            data_for_condition = concatenated_data[condition_trials]
            
            # Count total trials for the current condition
            trial_counts[condition_label] = np.sum(condition_trials)
            
            # Count NaNs per channel
            nan_counts_per_channel[condition_label] = np.isnan(data_for_condition).sum(axis=0)
            print(f'Condition {string_group} has {trial_counts[condition_label]} trials')
            print(f'NaN counts per channel for {string_group}: {nan_counts_per_channel[condition_label]}')

        # Determine the maximum trial count across conditions
        max_trial_count = max(trial_counts.values())
        
        # Balance trial counts by adding NaN trials where needed
        for condition_label, count in trial_counts.items():
            # Calculate the number of trials to add for this condition
            trials_to_add = max_trial_count - count
            
            if trials_to_add > 0:
                # Create NaN trials with the same shape as `concatenated_data` and add them
                nan_trial_shape = (trials_to_add,) + concatenated_data.shape[1:]
                nan_trials = np.full(nan_trial_shape, np.nan)
                
                # Append NaN trials to `concatenated_data` and update `labels` accordingly
                concatenated_data = np.concatenate([concatenated_data, nan_trials], axis=0)
                labels = np.concatenate([labels, [condition_label] * trials_to_add])
        
        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)

        # Use the concatenated data for the decoder
        cm = decoder.cv_cm_jim(concatenated_data, labels, normalize='true', obs_axs=obs_axs)
        cm_avg = np.mean(cm, axis=0)

        # Store the confusion matrix in the dictionary
        confusion_matrices[roi] = cm_avg

        # Convert tuple labels to simple strings for display
        display_labels = [
            key[0] if isinstance(key, tuple) and len(key) == 1 else str(key)
            for key in cats.keys()
        ]
        
        # Plot the Confusion Matrix
        fig, ax = plt.subplots()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=display_labels)
        disp.plot(ax=ax)

        # Save the figure with the time interval in the filename
        time_str = f"_{time_interval_name}" if time_interval_name else ""
        file_name = f'{roi}_{condition_comparison}{time_str}_time_averaged_confusion_matrix_{n_splits}splits_{n_repeats}repeats.png'
        plt.savefig(os.path.join(save_dir, file_name))
        plt.close()

    return confusion_matrices


In [86]:
# Call the old cm function with the appropriate parameters
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,  # If not dealing with specific time intervals
        n_splits=5,
        n_repeats=10,
        obs_axs=0
    )

Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 641)


ValueError: Need at least 3 non-nan values, but only have 0

prestimulus vs poststimulus confusion matrices 9/21

In [68]:
def get_data_in_time_range(labeled_array, time_range):
    """
    Extract data from a LabeledArray where the time points fall within a given range, using the LabeledArray `take` function.
    
    Parameters:
    - labeled_array: The LabeledArray containing time points as labels.
    - time_range: A tuple (start_time, end_time) representing the range of time.
    
    Returns:
    - filtered_data: A LabeledArray containing only the data where the timepoints are within the specified range.
    """
    start_time, end_time = time_range

    # Assume that the time labels are stored in the 4th dimension (index 3)
    time_points = np.array(labeled_array.labels[3], dtype=float)  # convert to floats, ensure conversion to float numpy array

    # Find the indices of time points within the specified range
    time_indices = np.where((time_points >= start_time) & (time_points <= end_time))[0]

    # Use the take function to select the time indices along the time axis (axis=3)
    filtered_data = labeled_array.take(time_indices, axis=3)

    return filtered_data


In [31]:
# Define the time intervals of interest
pre_stimulus_interval = (-1.0, 0.0)  # Pre-stimulus: time -1 to 0 seconds
post_stimulus_interval = (0.0, 1.5)  # Post-stimulus: time 0 to 1.5 seconds
pre_stimulus_roi_labeled_arrays = {}
post_stimulus_roi_labeled_arrays = {}

for roi, labeled_array in roi_labeled_arrays.items():
    pre_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, pre_stimulus_interval)
    post_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, post_stimulus_interval)


pre_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays=pre_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='pre_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

post_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays=post_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='post_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: v1
Concatenated data shape for v1: (472, 11, 257)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: occ
Concatenated data shape for occ: (512, 51, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (512, 23, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (472, 10, 257)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 385)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: v1
Concatenated data shape for v1: (472, 11, 385)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: occ
Concatenated data shape for occ: (512, 51, 385)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (51

define colors for plotting (not used yet as of 8/21)

In [32]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}

test this for stats though i think should be an easier way using the get_scores with shuffle=True. 11/1 don't use this, try using aaron's methods.

In [ ]:
import numpy as np
from scipy.stats import norm
from ieeg.calc.stats import time_perm_cluster
import matplotlib.pyplot as plt

def time_sliding_decoding_with_significance(decoder, x_data, labels, conds, n_shuffles=100, alpha=0.05):
    """
    Perform time-sliding decoding with significance testing using shuffled data.

    Parameters:
    - decoder: The Decoder object for performing decoding.
    - x_data: The data array of shape (trials, channels, timepoints).
    - labels: Array of trial labels.
    - conds: List of conditions or labels to decode.
    - n_shuffles: Number of shuffling iterations for significance testing.
    - alpha: Significance level for permutation testing.

    Returns:
    - true_scores: The decoding accuracy for the true labels.
    - shuffle_scores: Decoding accuracy across shuffles.
    - significance: Boolean array indicating significant time points.
    """
    # Step 1: Perform decoding with true labels
    true_scores = decoder.cv_cm(x_data, labels, normalize='true', obs_axs=0)

    # Step 2: Perform decoding with shuffled labels
    shuffle_scores = np.zeros((n_shuffles, true_scores.shape[-1]))  # (n_shuffles, timepoints)
    for i in range(n_shuffles):
        np.random.shuffle(labels)  # Shuffle the labels in-place
        shuffle_result = decoder.cv_cm(x_data, labels, normalize='true', obs_axs=0)
        shuffle_scores[i, :] = np.mean(shuffle_result, axis=(0, 1))  # Average across folds and repeats

    # Step 3: Statistical testing (e.g., cluster-based permutation test)
    true_mean = np.mean(true_scores, axis=(0, 1))  # Average across folds and repeats
    shuffle_mean = np.mean(shuffle_scores, axis=0)

    # Use permutation cluster testing to find significant time points
    significance, _ = time_perm_cluster(true_mean, shuffle_scores, alpha=alpha)

    return true_mean, shuffle_scores, significance

def plot_decoding_with_significance(true_scores, shuffle_scores, significance, conds, alpha=0.05):
    """
    Plot the decoding accuracy with shaded regions indicating significance.

    Parameters:
    - true_scores: The mean decoding accuracy for the true labels.
    - shuffle_scores: The decoding accuracy for shuffled labels.
    - significance: Boolean array indicating significant time points.
    - conds: List of condition names.
    - alpha: Significance level used for statistical testing.
    """
    time_points = np.arange(true_scores.shape[-1])

    # Plot the true decoding scores
    plt.plot(time_points, true_scores, label='True Decoding Accuracy', color='b')

    # Plot the mean and standard deviation of shuffled scores
    shuffle_mean = np.mean(shuffle_scores, axis=0)
    shuffle_std = np.std(shuffle_scores, axis=0)
    plt.fill_between(time_points, shuffle_mean - shuffle_std, shuffle_mean + shuffle_std,
                     color='gray', alpha=0.5, label='Shuffled Decoding (Mean ± SD)')

    # Highlight significant time points
    plt.fill_between(time_points, 0, 1, where=significance, color='red', alpha=0.3, transform=plt.gca().get_xaxis_transform(),
                     label=f'Significant (p < {alpha})')

    # Plot settings
    plt.axhline(0.25, color='k', linestyle='--', label='Chance Level')
    plt.xlabel('Time Points')
    plt.ylabel('Decoding Accuracy')
    plt.title('Time-Sliding Decoding with Significance')
    plt.legend()
    plt.show()


In [ ]:
# Example usage:
if __name__ == '__main__':
    # Assuming `decoder`, `x_data`, and `labels` are defined
    # conds = ['condition1', 'condition2']  # Modify as needed
    true_scores, shuffle_scores, significance = time_sliding_decoding_with_significance(
        decoder, x_data, labels, conds, n_shuffles=100, alpha=0.05)

    # Plot the results
    plot_decoding_with_significance(true_scores, shuffle_scores, significance, conds, alpha=0.05)


10/29 use aarons init cv cm code that has the shuffle option

In [45]:
def get_and_plot_confusion_matrix_for_rois_aaron(roi_labeled_arrays, rois, strings_to_find, condition_0_name, condition_1_name, save_dir, time_interval_name=None, explained_variance=0.80, n_splits=5, n_repeats=5, n_shuffle_repeats=20, obs_axs=0):
    """
    Compute the confusion matrix for each ROI and return it. Does this using all time points for training.

    Parameters:
    - roi_labeled_arrays: Dictionary containing the reshaped data for each ROI.
    - rois: List of regions of interest (ROIs) to process.
    - strings_to_find: List of strings or string groups to identify conditions.
    - condition_0_name: Name for condition 0 (used for labeling).
    - condition_1_name: Name for condition 1 (used for labeling).
    - save_dir: Directory to save the confusion matrix plots.
    - time_interval_name: Optional string to add to the filename for the time interval (e.g., 'pre_stimulus' or 'post_stimulus').
    - obs_axs: the trials axis of your time binned arrays

    Returns:
    - confusion_matrices: Dictionary containing confusion matrices for each ROI.
    """
    confusion_matrices = {}

    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Concatenate the trials and get labels
        # concatenated_data, labels, cats = concatenate_conditions_by_string(reshaped_roi_labeled_arrays_for_decoding, roi, strings_to_find)
        concatenated_data, labels, cats = concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find) # don't reshape

        print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")
        
        # Iterate over the groups in strings_to_find
        for string_group in strings_to_find:
            # If it's a single string (old behavior)
            if isinstance(string_group, str):
                # Count trials for the single condition group
                num_trials = len([label for label in labels if label == cats[(string_group,)]])
                print(f"{string_group} has {num_trials} trials")
            
            # If it's a list of strings (new behavior)
            elif isinstance(string_group, list):
                # Count trials for the group of strings (e.g., ['i25.0', 'i75.0'])
                num_trials = len([label for label in labels if label == cats[tuple(string_group)]])
                print(f"{string_group} has {num_trials} trials")

        # Create a Decoder and run cross-validation. Check if Aaron makes a new Decoder for the shuffled labels.
        decoder = Decoder(cats, explained_variance, oversample=True, n_splits=n_splits, n_repeats=n_repeats)
        decoder_shuffled = Decoder(cats, explained_variance, oversample=True, n_splits=n_splits, n_repeats=n_shuffle_repeats)

        # Use the concatenated data for the decoder using aaron's cv cm function with the shuffle option
        cm = decoder.cv_cm_aaron(concatenated_data, labels, normalize='true', shuffle=False, obs_axs=obs_axs)
        cm_shuffle = decoder_shuffled.cv_cm_aaron(concatenated_data, labels, normalize='true', shuffle=True, obs_axs=obs_axs)
        cm_avg = np.mean(cm, axis=0)
        cm_shuffle_avg = np.mean(cm_shuffle, axis=0)

        # Store the real and shuffle confusion matrices as a dictionary
        confusion_matrices[roi] = {'real': cm_avg, 'shuffle': cm_shuffle_avg}
        
        # Plot both cm_avg and cm_shuffle_avg for comparison
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        display_labels = [
            key[0] if isinstance(key, tuple) and len(key) == 1 else str(key)
            for key in cats.keys()
        ]

        # Plot original confusion matrix
        disp_real = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=display_labels)
        disp_real.plot(ax=axs[0], cmap="Blues", values_format=".2f")
        axs[0].set_title(f"{roi} Confusion Matrix (Real)")

        # Plot shuffled confusion matrix
        disp_shuffle = ConfusionMatrixDisplay(confusion_matrix=cm_shuffle_avg, display_labels=display_labels)
        disp_shuffle.plot(ax=axs[1], cmap="Oranges", values_format=".2f")
        axs[1].set_title(f"{roi} Confusion Matrix (Shuffled)")

        # Save the figure
        time_str = f"_{time_interval_name}" if time_interval_name else ""
        file_name = f'{roi}_{condition_0_name}_vs_{condition_1_name}{time_str}_time_averaged_confusion_matrix_comparison_{n_splits}splits_{n_repeats}repeats.png'
 
        plt.savefig(os.path.join(save_dir, file_name))
        plt.show()

    return confusion_matrices


In [46]:
# Call the new function with the appropriate parameters
confusion_matrices = get_and_plot_confusion_matrix_for_rois_aaron(
    roi_labeled_arrays=roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name=None,  # If not dealing with specific time intervals
    explained_variance=0.80,
    n_splits=5,
    n_repeats=10,
    n_shuffle_repeats=20,
    obs_axs=0
)

Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 641)
Taskg has 256 trials
Taskl has 256 trials


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'